## tinyFrame Analysis - General Calculations

## Purpose
This document serve as the repository for all analysis for the tinyFrame project.

## TODO:
- ~~Calculate required deflection~~
- ~~Build calculations around known press-force (7 lbf)~~
- Stress checks for given thickness and the declarations above

In [1]:
import numpy, yaml, pint
import numpy as np
import matplotlib.pyplot as plt

# Boilerplate
## Initialize Pint Registry for Unit Manipulation
## See documentation on Registries and Units in Pint here:
## https://pint.readthedocs.io/en/stable/getting/tutorial.html
from pint import UnitRegistry
ureg = UnitRegistry(auto_reduce_dimensions=True)

In [2]:
preferred_units = [
   ureg.m,  # distance      L
   ureg.kilogram,  # mass          M
   ureg.s,  # duration      T
   ureg.c,  # temperature   Θ
   ureg.newton,  # force         L M T^-2
   ureg.W,  # power         L^2 M T^-3
]

Equation that describes deflection as a function of loading for wrap:
$$\delta = \frac{Pl^3}{3EI}$$

Equation for Second Moment of Area of a Rectangular beam:
$$I = \frac{bh^3}{12}$$

Before solving for the required locking tab length, we first need to find the prototyped deflection to determine the required length in thicker gauge configurations of the tubing system.

Solving the equation above and replacing the second moment of area, I with the locking tab dimensions, we get:

$$\delta = \frac{4 P L^3}{E b h^3}$$

We normalize to the number of tabs in the system (four) and end up with:


$$\delta = \frac{P L_{protoyped}^3}{E b h^3}$$




In [49]:
# Values determined from Prototyping (Thanks Jonathan & Bryce!)
E = 69e9 * ureg.newton / (ureg.meter * ureg.meter) # Pa - young's modulus
P = 31 * ureg.newton # From Prototyping
L_prototyped = 75 * ureg.mm # From CAD

# Second moment of Area for Beam
b = 6.4177 * ureg.mm
h = 1.651 * ureg.mm

In [50]:
del_ = (P * L_prototyped**3) / (E * b * h**3)

print("Distance for Locking Flexure to deflect for installation: ")
print(del_.to(ureg.mm))

Distance for Locking Flexure to deflect for installation: 
6.562597392958355 millimeter


Given the analysis matches CAD, we can utilize the required deflection as a function of the tab depth as opposed to what was regressed from the calculations above. That will be declared as del_tab.

We need to calculate the length of the flexure (locking tabs) required - solve the above equations for I:

$$L = \sqrt[3]{\frac{3 \delta \, E \, b \, h^3}{12P}}$$

P represents the global press force, but we assume there are always four tabs, so we divide P by 4 and are left with:

$$L = \sqrt[3]{\frac{\delta \, E \, b \, h^3}{P}}$$

In [73]:
# Declare variables required:
del_tab = 8 * ureg.mm # From CAD


# Determine the length of the locking tabs
L = ((del_tab * E * b * h**3) / (P)) ** (1/3)

#print(L_temp.to(ureg.mm))
print(L)

80.11847456328006 millimeter


Now, we check for bending stress:

$$\sigma_b = \frac{M \cdot c}{I}$$

In [74]:
# Declare variables
sigma_yield = 73 * ureg.ksi
c = h/2
I = b*h**3 / 12

# Calculate requisite components
M = P/4 * L

sigma_bending =  M * c / I

# What's the factor of safety against the material yield stress?
FOS_install = sigma_yield / sigma_bending

print("Bending stress seen by the locking tab: ")
print(sigma_bending.to(ureg.ksi))

print("Factor of safety against yield: ")
print(FOS_install)

Bending stress seen by the locking tab: 
30.888234338738997 kip_per_square_inch
Factor of safety against yield: 
2.3633594332210133 dimensionless
